In [ ]:
%matplotlib inline
import re
#import matplotlib
import numpy as np
#import matplotlib.pyplot as plt
import pandas as pd
#from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.naive_bayes import MultinomialNB
#from sklearn.metrics import accuracy_score
#from sklearn.multiclass import OneVsRestClassifier
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
#stop_words = set(stopwords.words('english'))
from sklearn.svm import LinearSVC
#from sklearn.linear_model import LogisticRegression
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.linear_model import PassiveAggressiveClassifier
#from sklearn.pipeline import Pipeline
from sklearn.feature_selection import chi2
#import seaborn as sns
import unidecode


In [ ]:
stopwords_cust = stopwords.words('portuguese')

df_swc = pd.read_excel('C:\Users\marcel.massa.LGE\ML\stop_words_customized.xlsx')

# STOP_WORDS_CUSTOMIZED is the name of the column in the Excel file

df_swc.STOP_WORDS_CUSTOMIZED = df_swc.STOP_WORDS_CUSTOMIZED.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()
for count in range (df_swc.shape[0]):
    stopwords_cust.append(df_swc.STOP_WORDS_CUSTOMIZED[count])


In [ ]:
# Opening training data

df_TD = pd.read_excel('C:\\Users\\marcel.massa.LGE\\ML\\train_multilabel_NEW.xlsx')

In [ ]:
# Creating list of Symptoms (categories - the lables of multi-label)
# This is needed further ahead, but it could be done in a smarter way.

df_symptoms = df_TD.drop(['Claim'], axis=1)
categories = list(df_symptoms.columns.values)

In [ ]:
#Vectorizing the text values (Claim) from Training Data

vectorizer = TfidfVectorizer(sublinear_tf=True, min_df=5, encoding='utf-8', strip_accents='unicode', ngram_range=(1,2), norm='l2', lowercase=True, stop_words=stopwords_cust)

features = vectorizer.fit_transform(df_TD.Claim)
print(features.shape)

In [ ]:
#Opening the data to be predicted

df_Data = pd.read_excel('C:\Users\marcel.massa.LGE\ML\CIC Call Receiving Rate_120515012.xls', header=1)
df_Data.head()

In [ ]:
#Vectorizing the text values (Consultation Content) from new Data

features_Data = vectorizer.transform(df_Data.loc[: ,"Consultation Content"])
print(features_Data.shape)

In [ ]:
# If you're not using it for Power BI, SKIP THIS! It takes a while to run...

# Creating a series with "clean text" (tokenize, lowercase, strip accents, remove stopwords, de-tokenize)
# to be added to final Excel file and used in Word Cloud of Power BI dashboard.

list_tokens = pd.Series(df_Data.loc[: ,"Consultation Content"], name='Tokens').str.lower().tolist()
list_CleanText = []

for string in list_tokens: 
    tokens = word_tokenize(unidecode.unidecode(string))
    clean_tokens = [w for w in tokens if not w in stopwords_cust]
    list_CleanText.append(TreebankWordDetokenizer().detokenize(clean_tokens))

series_CleanText = pd.Series(list_CleanText, name='Texto_Limpo')
print('... [DONE]')

In [ ]:
from skmultilearn.problem_transform import ClassifierChain
classifier = ClassifierChain(LinearSVC())

# Training LinearSVC model on training data
classifier.fit(features, df_TD.drop(labels = ['Claim'], axis=1))

# Predicting
predictions = classifier.predict(features_Data).toarray()
predictions.shape

# Turning predictions array into a dataframe, to be concatenated to original dataframe with the source data.
df_predictions = pd.DataFrame(predictions, columns=categories)
df_predictions.head()

In [ ]:
# Concatenating dataframes (original, clean text, predictions)

df_Final = pd.concat([df_Data, series_CleanText.to_frame(), df_predictions], axis=1)
df_Final.head()

In [ ]:
# Recording the final dataframe to a new Excel file.
# This is the end of the classification process.

df_Final.to_excel('C:\Users\marcel.massa.LGE\ML\CIC Call Receiving Rate_MULTI_LABEL_2_PREDICTED.xlsx')

In [ ]:
# OPTIONAL STEP
# This is only to analyze the most common unigrams and bigrams for each symptom, from the training dataset.

N = 5
for i in categories:
    features_chi2 = chi2(features, df_TD[i] )
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(vectorizer.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("# '{}':".format(i))
    print("  . Most correlated unigrams:\n       . {}".format('\n       . '.join(unigrams[-N:]).encode('utf-8')))
    print("  . Most correlated bigrams:\n       . {}".format('\n       . '.join(bigrams[-N:]).encode('utf-8')))